<a href="https://colab.research.google.com/github/LonghuaHu/deep-learning/blob/master/WCNN_100.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
      from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [2]:
import numpy as np
from sklearn.model_selection import train_test_split
import h5py 
#data
f = h5py.File('/content/drive/My Drive/Colab Notebooks/catdog/waveletImages.mat','r') 

X = f['waveletImages'][:]
#data = data.T
print(X.shape)

#lable
#f = h5py.File('/content/drive/My Drive/Colab Notebooks/new_quaternionImagesLabels.mat','r') 
y = f['waveletImagesLabels'][:]
y = y.T
print(y.shape)




(25000, 4, 75, 75)
(25000, 1)


In [0]:
X_train, X_test, y_train, y_test = train_test_split(
X, y, test_size=0.996, random_state=42)

In [4]:
X_train = X_train.reshape(X_train.shape[0], X_train.shape[1], X_train.shape[2], X_train.shape[3],1)
X_test = X_test.reshape(X_test.shape[0], X_train.shape[1], X_train.shape[2], X_train.shape[3],1)
#training_labels = training_labels[0]
y_train.astype(int)
y_test.astype(int)
print('Number of training examples: ' + str(X_train.shape[0]))
print('Training images shape:' + str(X_train.shape))
print('Training labels shape:' + str(y_train.shape))
print('Test images shape:' + str(X_test.shape))
print('Test labels shape:' + str(y_test.shape))

Number of training examples: 100
Training images shape:(100, 4, 75, 75, 1)
Training labels shape:(100, 1)
Test images shape:(24900, 4, 75, 75, 1)
Test labels shape:(24900, 1)


In [5]:
import os
import tensorflow as tf
from keras.callbacks import TensorBoard

Using TensorFlow backend.


In [6]:
model = tf.keras.models.Sequential()
model.add(tf.keras.layers.Conv3D(32, (1,3,3), activation='relu', input_shape=(4, 75, 75, 1)))
#model.add(tf.keras.layers.MaxPooling3D(pool_size=(1, 2, 2), strides=(1, 2, 2)))
#model.add(tf.keras.layers.BatchNormalization())
model.add(tf.keras.layers.Conv3D(64, (1,3,3), activation='relu'))
model.add(tf.keras.layers.MaxPooling3D(pool_size=(1, 2, 2), strides=(1, 2, 2)))
model.add(tf.keras.layers.Conv3D(128, (1,3,3), activation='relu'))
model.add(tf.keras.layers.MaxPooling3D(pool_size=(1, 2, 2), strides=(1, 2, 2)))
model.add(tf.keras.layers.Conv3D(128, (1,3,3), activation='relu'))
model.add(tf.keras.layers.MaxPooling3D(pool_size=(1, 2, 2), strides=(1, 2, 2)))
model.add(tf.keras.layers.BatchNormalization())


#FC Layers
model.add(tf.keras.layers.Flatten())
model.add(tf.keras.layers.Dense(512))
model.add(tf.keras.layers.Dense(1, activation='sigmoid'))

model.summary()



W0905 02:14:19.620139 140509921568640 deprecation.py:506] From /usr/local/lib/python3.6/dist-packages/tensorflow/python/ops/init_ops.py:1251: calling VarianceScaling.__init__ (from tensorflow.python.ops.init_ops) with dtype is deprecated and will be removed in a future version.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor


Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv3d (Conv3D)              (None, 4, 73, 73, 32)     320       
_________________________________________________________________
conv3d_1 (Conv3D)            (None, 4, 71, 71, 64)     18496     
_________________________________________________________________
max_pooling3d (MaxPooling3D) (None, 4, 35, 35, 64)     0         
_________________________________________________________________
conv3d_2 (Conv3D)            (None, 4, 33, 33, 128)    73856     
_________________________________________________________________
max_pooling3d_1 (MaxPooling3 (None, 4, 16, 16, 128)    0         
_________________________________________________________________
conv3d_3 (Conv3D)            (None, 4, 14, 14, 128)    147584    
_________________________________________________________________
max_pooling3d_2 (MaxPooling3 (None, 4, 7, 7, 128)      0

In [7]:
import matplotlib.pyplot as plt
from tensorflow.keras.optimizers import RMSprop

model.compile(loss='binary_crossentropy',
              optimizer=RMSprop(lr=0.0003),
              metrics=['acc'])

history_wcnn = model.fit(X_train, y_train, epochs=20, verbose=1)
test_loss = model.evaluate(X_test, y_test)


#model.fit(X_train, y_train, epochs=30)
#test_loss = model.evaluate(X_test, y_test)

#model.fit(X_train, y_train, epochs=10,
#          validation_data=(X_test, y_test),
#          callbacks=[TrainValTensorBoard(write_graph=False)])

W0905 02:14:23.427103 140509921568640 deprecation.py:323] From /usr/local/lib/python3.6/dist-packages/tensorflow/python/ops/nn_impl.py:180: add_dispatch_support.<locals>.wrapper (from tensorflow.python.ops.array_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


Epoch 1/20
100/100 [==============================] - 6s 55ms/sample - loss: 31.0840 - acc: 0.5600
Epoch 2/20
100/100 [==============================] - 0s 3ms/sample - loss: 17.4555 - acc: 0.5900
Epoch 3/20
100/100 [==============================] - 0s 3ms/sample - loss: 0.6508 - acc: 0.8900
Epoch 4/20
100/100 [==============================] - 0s 3ms/sample - loss: 10.4410 - acc: 0.5700
Epoch 5/20
100/100 [==============================] - 0s 3ms/sample - loss: 0.6104 - acc: 0.9100
Epoch 6/20
100/100 [==============================] - 0s 3ms/sample - loss: 0.0702 - acc: 0.9700
Epoch 7/20
100/100 [==============================] - 0s 3ms/sample - loss: 0.0130 - acc: 0.9900
Epoch 8/20
100/100 [==============================] - 0s 3ms/sample - loss: 2.4728 - acc: 0.8200
Epoch 9/20
100/100 [==============================] - 0s 3ms/sample - loss: 0.3174 - acc: 0.9600
Epoch 10/20
100/100 [==============================] - 0s 3ms/sample - loss: 0.3111 - acc: 0.9400
Epoch 11/20
100/100 [====